In [ ]:
import warnings
from numpy import nan, inf, exp, log
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Metatrader_WINN_M15 (After Covid).csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['M15'])
Query.head()

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]          = Query[['a','z']]
    Src[['Symbol','TF']]    = Query[['symbol','tf']]

    Src['Datetime']         = Query['datetime']
    Src['Date']             = Query['datetime'].dt.date
    Src['Time']             = Query['datetime'].dt.time
    Src['Time%']            = Query['datetime'].dt.time.apply(lambda x: x.hour + x.minute/60)

    Src_by2                 = Src.groupby(['Symbol','TF'],        sort=0, group_keys=0)
    Src_by3                 = Src.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Src['D']                = (+1) * Src_by2['Date'].rank(method='dense')
    Src['T']                = (-1) * Src_by3['Time'].rank(method='dense', ascending=False)
    
    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

# Data

In [ ]:
def CALCULATIONS(Src):
    # ================ Helpers ================ #
    def _over(Df, Ref, val, Col, get, alt=nan):
        try:    return Df[Df[Ref]==val][Col].iloc[get]
        except: return alt

    # ================ Main ================ #
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)


    Calc['Day Open']        = Calc_by['Open']   .transform('first')
    Calc['Day High']        = Calc_by['High']   .transform('max')
    Calc['Day Low']         = Calc_by['Low']    .transform('min')
    Calc['Day Close']       = Calc_by['Close']  .transform('last')

    Calc['Day Change']      = (Calc['Day Close'] - Calc['Day Open'])
    Calc['Day Variat']      = (Calc['Day Close'] / Calc['Day Open']) *100-100
    Calc['Day Hilo']        = (Calc['Day High']  - Calc['Day Low'])

    Calc['Day High Rnk']    = Calc_by['High']  .rank(ascending=False)
    Calc['Day Low Rnk']     = Calc_by['Low']   .rank(ascending=True)


    Calc['Prev Open']       = Calc.apply(lambda row: _over(Calc, Ref='D', val=(row['D']-1), Col='Day Open',  get=(-1)), axis=1)
    Calc['Prev High']       = Calc.apply(lambda row: _over(Calc, Ref='D', val=(row['D']-1), Col='Day High',  get=(-1)), axis=1)
    Calc['Prev Low']        = Calc.apply(lambda row: _over(Calc, Ref='D', val=(row['D']-1), Col='Day Low',   get=(-1)), axis=1)
    Calc['Prev Close']      = Calc.apply(lambda row: _over(Calc, Ref='D', val=(row['D']-1), Col='Day Close', get=(-1)), axis=1)

    Calc['Prev Change']     = (Calc['Prev Close'] - Calc['Prev Open'])
    Calc['Prev Variat']     = (Calc['Prev Close'] / Calc['Prev Open']) *100-100
    Calc['Prev Hilo']       = (Calc['Prev High']  - Calc['Prev Low'])


    Calc['ID High Cumax']   = Calc_by['High'] .expanding().max()      .reset_index(drop=1)
    Calc['ID Low Cumin']    = Calc_by['Low']  .expanding().min()      .reset_index(drop=1)

    Calc['Test Up']         = (Calc['High']    - Calc['Prev High']) .apply(lambda x: 0 if (x < 0) else x)
    Calc['Test Down']       = (Calc['Prev Low'] - Calc['Low'])      .apply(lambda x: 0 if (x < 0) else x)

    Calc['ID Test Up Rnk']   = Calc_by['Test Up']    .rank(method='dense').sub(1)
    Calc['ID Test Down Rnk'] = Calc_by['Test Down']  .rank(method='dense').sub(1)
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

In [ ]:
# Calc.to_csv('Cache/Calc.csv', index=False)
# Calc = pd.read_csv('Cache/Calc.csv')
# Calc.head()

In [ ]:
BINS_DIF = [500,700,1000,1300,1600,1900,2200,2500,2800,3100,3500,4000,4500,5000,inf]
BINS_VAR = [-inf, -5.00, -3.00, -2.00, -1.50, -1.00, -0.75, -0.50, -0.25, 0, +0.25, +0.50, +0.75, +1.00, +1.50, +2.00, +3.00, +5.00, +inf]

In [ ]:
def SIGNALS(Calc):
    # ================ Helpers ================ #
    def _int(x):
        try:    return int(x)
        except: return x

    # ================ Main ================ #
    Sign = pd.DataFrame(Calc)
    Sign['True']  = True
    Sign['False'] = False


    Sign['Day Is Bull']  =  Sign['Day Change'].apply(lambda x: x >= 0)
    Sign['Day Is Bear']  =  Sign['Day Change'].apply(lambda x: x <  0)

    Sign['Prev Is Bull'] = Sign['Prev Change'].apply(lambda x: x >= 0)
    Sign['Prev Is Bear'] = Sign['Prev Change'].apply(lambda x: x <  0)



    for (a,b) in [win for win in pd.Series(BINS_DIF).rolling(2) if len(win) >= 2]:
        Sign[f'Prev Hilo {_int(a)}-{_int(b)}'] = (a < Sign['Prev Hilo']) & (Sign['Prev Hilo'] < b)


    for (a,b) in [win for win in pd.Series(BINS_VAR).rolling(2) if len(win) >= 2]:
        Sign[f'Prev Variat {a}  –  {b}'] = (a < Sign['Prev Variat']) & (Sign['Prev Variat'] < b)



    for OHLC in ['Open','High','Low','Close']:
        Sign[f'{OHLC} Out Max'] =                                                  (             Sign['Prev High'] < Sign[OHLC])
        Sign[f'{OHLC} In Max']  =                                                  (Sign[OHLC] < Sign['Prev High']             )
        Sign[f'{OHLC} Inside']  = (             Sign['Prev Low'] < Sign[OHLC])  &  (Sign[OHLC] < Sign['Prev High']             )
        Sign[f'{OHLC} In Min']  = (             Sign['Prev Low'] < Sign[OHLC])
        Sign[f'{OHLC} Out Min'] = (Sign[OHLC] < Sign['Prev Low']             )


    for OHLC in ['Open','High','Low','Close']:
        Sign[f'Day {OHLC} Out Max'] =                                                                    (                      Sign['Prev High'] < Sign[f'Day {OHLC}'])
        Sign[f'Day {OHLC} In Max']  =                                                                    (Sign[f'Day {OHLC}'] < Sign['Prev High']                      )
        Sign[f'Day {OHLC} Inside']  = (                      Sign['Prev Low'] < Sign[f'Day {OHLC}'])  &  (Sign[f'Day {OHLC}'] < Sign['Prev High']                      )
        Sign[f'Day {OHLC} In Min']  = (                      Sign['Prev Low'] < Sign[f'Day {OHLC}'])
        Sign[f'Day {OHLC} Out Min'] = (Sign[f'Day {OHLC}'] < Sign['Prev Low']                      )

    return Sign

Sign = SIGNALS(Calc)
Sign.head()

# Stats

In [ ]:
def STATS(Sign, BINS, conds, rank):
    # ================ Helpers ================ #
    def _to_clock(x): return round(x//1 + x%1 * 60/100, 2)

    # ================ Main ================ #
    pipe = []
    for b in BINS: 
        Df = Sign[Sign[b] & Sign[conds].all(1) & (Sign[rank]==1)]
        pipe.append({ 'Condition':b, **Df['Time%'].describe().apply(_to_clock).fillna('') }) 
    pass
    return pd.DataFrame(pipe)

# Testes e Romps (Dias Pos)

In [ ]:
BINS = Sign.loc[:, f'Prev Hilo 500-700':f'Prev Hilo 5000-inf'].columns

Horário Médio que fez máxima do dia

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bull'], rank='Day High Rnk')

Horário Médio que fez mínima do dia

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bull'], rank='Day Low Rnk')

Testou a mínima no horário médio de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'Low Out Min', 'Day Close In Min'], rank='ID Test Down Rnk')

Testou a máxima no horário médio de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'High Out Max', 'Day Close In Max'], rank='ID Test Up Rnk')

Fechou abaixo da mínima de ontem e fez máxima na hora média de

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'High Out Max', 'Day Close Out Min'], rank='Day High Rnk')

Fechou abaixo da mínima de ontem e fez mínima na hora média de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'Low Out Min', 'Day Close Out Min'], rank='Day Low Rnk')

Fechou acima da máxima de ontem e fez mínima na hora média de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'Low Out Min', 'Day Close Out Max'], rank='Day Low Rnk')

Fechou acima da máxima de ontem e fez máxima na hora média de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bull', 'High Out Max', 'Day Close Out Max'], rank='Day High Rnk')

# Testes e Romps (Dias Neg)

Horário Médio que fez máxima do dia

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bear'], rank='Day High Rnk')

Horário Médio que fez mínima do dia

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bear'], rank='Day Low Rnk')

Testou a mínima no horário médio de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'Low Out Min', 'Day Close In Min'], rank='ID Test Down Rnk')

Testou a máxima no horário médio de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'High Out Max', 'Day Close In Max'], rank='ID Test Up Rnk')

Fechou abaixo da mínima de ontem e fez máxima na hora média de

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'High Out Max', 'Day Close Out Min'], rank='Day High Rnk')

Fechou abaixo da mínima de ontem e fez mínima na hora média de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'Low Out Min', 'Day Close Out Min'], rank='Day Low Rnk')

Fechou acima da máxima de ontem e fez mínima na hora média de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'Low Out Min', 'Day Close Out Max'], rank='Day Low Rnk')

Fechou acima da máxima de ontem e fez máxima na hora média de:

In [ ]:
STATS(Sign, BINS, conds=['Prev Is Bear', 'High Out Max', 'Day Close Out Max'], rank='Day High Rnk')

# Testes e Romps (Variat)

In [ ]:
BINS = Sign.loc[:, f'Prev Variat -inf  –  -5.0':f'Prev Variat 5.0  –  inf'].columns[::-1]

Horário Médio que fez máxima do dia

In [ ]:
STATS(Sign, BINS, conds=['True'], rank='Day High Rnk')

Horário Médio que fez mínima do dia

In [ ]:
STATS(Sign, BINS, conds=['True'], rank='Day Low Rnk')

Testou a mínima no horário médio de:

In [ ]:
STATS(Sign, BINS, conds=['Low Out Min', 'Close In Min'], rank='ID Test Down Rnk')

Testou a máxima no horário médio de:

In [ ]:
STATS(Sign, BINS, conds=['High Out Max', 'Close In Max'], rank='ID Test Up Rnk')

Fechou abaixo da mínima de ontem e fez máxima na hora média de

In [ ]:
STATS(Sign, BINS, conds=['High Out Max', 'Day Close Out Min'], rank='Day High Rnk')

Fechou abaixo da mínima de ontem e fez mínima na hora média de:

In [ ]:
STATS(Sign, BINS, conds=['Low Out Min', 'Day Close Out Min'], rank='Day Low Rnk')

Fechou acima da máxima de ontem e fez mínima na hora média de:

In [ ]:
STATS(Sign, BINS, conds=['Low Out Min', 'Day Close Out Max'], rank='Day Low Rnk')

Fechou acima da máxima de ontem e fez máxima na hora média de:

In [ ]:
STATS(Sign, BINS, conds=['High Out Max', 'Day Close Out Max'], rank='Day High Rnk')